# EEP 153 Proj 2: Group Sylvia Lane

## Minimum Cost Diet

**Topic**: Cost comparison of a minimized vegetarian diet for Berkeley residents across 5 different local grocery stores (shopping at one among Safeway, Trader Joe's, Amazon Fresh, Berkeley Bowl, Sprout's)

**Objectives**:
- A
- B
- C

## Import Libraries & Read Datasets

In [2]:
# Install packages
!pip install -r requirements.txt
!pip install eep153_tools
!pip install gspread_pandas

# Import libraries
import numpy as np
import pandas as pd
import fooddatacentral as fdc
from eep153_tools.sheets import read_sheets

# API Key
apikey = "ztQPaunUYOUORb4qgWsBdjS2VDpKaYrwYmMP9LXV"

In [3]:
# Read diet minimum and maximum data
diet_min = pd.read_excel("Dietary Requirements.xlsx",sheet_name='diet_minimums',index_col=0)
diet_min

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
Energy,---,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,RDA,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",---,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",RDA,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",RDA,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",RDA,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",RDA,7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",RDA,80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,RDA,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


In [4]:
diet_max = pd.read_excel("Dietary Requirements.xlsx",sheet_name='diet_maximums',index_col=0)
diet_max

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
"Sodium, Na",UL,1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300
Energy,NaN,2500,2500,2500,2800,3000,3100,3100,3100,3100,3100,3100,3100,3100


### [A] Description of population of interest

Our group are interested in minimum cost diet for **diagnosed diabetes patients in Berkeley**.

### [A] Dietary Reference Intakes
Write a function that takes as arguments the characteristics of a person (e.g., age, sex) and returns a pandas.Series of Dietary Reference Intakes (DRI's) or Recommended Daily Allowances(RDA) of a variety of nutrients appropriate for your population of interest

In [5]:
def dietary_ref_intake(age,sex):
    """Takes in age and sex, and returns the dietary reference intake for the chosen population"""

    if age <= 3:
        col = 'C 1-3'
    age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(50,100)]
    for age_range in age_ranges:
        if age >= age_range[0] and age <= age_range[1]:
            col = sex + ' ' + str(age_range[0]) + '-' + str(age_range[1])
    return pd.Series(diet_min[col])  

In [6]:
dietary_ref_intake(age=22,sex='M')

Nutrition
Energy                            2400.0
Protein                             56.0
Fiber, total dietary                33.6
Folate, DFE                        400.0
Calcium, Ca                       1000.0
Carbohydrate, by difference        130.0
Iron, Fe                             8.0
Magnesium, Mg                      400.0
Niacin                              16.0
Phosphorus, P                      700.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      90.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)          120.0
Zinc, Zn                            11.0
Name: M 19-30, dtype: float64

### [A] Data on prices for different foods
Construct a google spreadsheet of different prices of different kinds of food. The different foods should be keyed to the USDA's Food Data Central database (https://fdc.nal.usda.gov/)

In [37]:
food = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRDIOWlxta4m4S6RldUFn2srNWYSCo9WeNh3CBKIrdpW9c6uJuxqVcBJzsxlRxTNOaAndUq9M1Ksscw/pub?output=xlsx'
df = pd.read_excel(food,sheet_name='Whole Foods')
df = df.iloc[:,:2].dropna(subset=['FDC'])
df = df.reset_index(drop=True)
df['FDC'] = df['FDC'].astype(int)
df

,Food,FDC
0,"Spinach, raw",11457
1,"Mushrooms, shiitake, raw",11238
2,"Carrots, raw",11124
3,"Squash, summer, zucchini, includes skin, raw",11477
4,"Broccoli, raw",11090
5,"Tomatoes, red, ripe, raw, year round average",11529
6,"Cabbage, red, raw",11112
7,"Avocados, raw, California",9038
8,"Blueberries, raw",9050
9,"Grapefruit, raw, white, California",9117


### [A] Nutritional content of different foods
For all the foods you're considering you'll need to be able to describe their nutritional content, in terms that allow you to compare with recommended daily allowances. 

In [39]:
import fooddatacentral as fdc
import warnings
apikey = "ztQPaunUYOUORb4qgWsBdjS2VDpKaYrwYmMP9LXV"
D = {}
count = 0
for food in  df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)
FoodNutrients

/var/folders/gh/rk3t_9fx5rv7y72f7k32rj7r0000gn/T/ipykernel_9220/2186797018.py:12: UserWarning: Couldn't find FDC Code Spinach, raw for food 11457.
  warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
/var/folders/gh/rk3t_9fx5rv7y72f7k32rj7r0000gn/T/ipykernel_9220/2186797018.py:12: UserWarning: Couldn't find FDC Code Mushrooms, shiitake, raw  for food 11238.
  warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
/var/folders/gh/rk3t_9fx5rv7y72f7k32rj7r0000gn/T/ipykernel_9220/2186797018.py:12: UserWarning: Couldn't find FDC Code Carrots, raw for food 11124.
  warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
/var/folders/gh/rk3t_9fx5rv7y72f7k32rj7r0000gn/T/ipykernel_9220/2186797018.py:12: UserWarning: Couldn't find FDC Code Squash, summer, zucchini, includes skin, raw for food 11477.
  warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))
/var/folders/gh/rk3t_9fx5rv7y72f7k32rj7r0000gn/T/ipykernel_9220/2186797018.py:12: U

""


In [44]:
fdc.nutrients(apikey,fdc_id=539572)

,Quantity,Units
"Calcium, Ca",41.00,mg
"Vitamin C, total ascorbic acid",89.20,mg
"Fatty acids, total saturated",0.00,g
"Iron, Fe",0.73,mg
Cholesterol,0.00,mg
"Vitamin A, IU",203.00,IU
"Sugars, total including NLEA",1.35,g
"Sodium, Na",54.00,mg
"Carbohydrate, by difference",5.41,g
"Fiber, total dietary",2.00,g


### [A] Solution

### [B] Is your solution edible?

### [B] Meal Reviews
